In [10]:
# import libraries
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

## Obteniendo datos

In [38]:
symbol = 'GOOGL'

url_bs = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
url_is = 'https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol
url_cf = 'https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p='+ symbol

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

def get_table(url):
    # Fetch the page that we're going to parse, using the request headers defined above
    page = requests.get(url, headers)

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Smoke test that we fetched the page by fetching and displaying the H1 element
    tree.xpath("//h1/text()")
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0

    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    df = pd.DataFrame(parsed_rows)
    df_org = df
    
    df = pd.DataFrame(parsed_rows)
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    df_rot = df
    return df_org, df_rot



# get Balance Sheet 
BS_orginal, BS_transpose = get_table(url_bs)

# get Income Statement 
IS_orginal, IS_transpose = get_table(url_is)


# get Cash Flow
CF_orginal, CF_transpose = get_table(url_cf)

BS_transpose.to_excel("google_bs.xlsx")
IS_transpose.to_excel("google_is.xlsx")
CF_transpose.to_excel("google_cf.xlsx")

## Analisis: Balance Sheet

In [55]:

BS_analysis = pd.DataFrame(BS_transpose['Date']) # copy columns of dataframe

# Working Capital
current_assets = BS_transpose['Total Current Assets'].str.replace(',', '').astype(int)
current_liabilities = BS_transpose['Total Current Liabilities'].str.replace(',', '').astype(int)
working_capital = current_assets - current_liabilities
BS_analysis['Working Capital'] = working_capital # copy columns of dataframe

#Current ratio
current_ratio = current_assets / current_liabilities
BS_analysis['Current Ratio'] = current_ratio


#Acid test

inventory = BS_transpose['Inventory'].str.replace(',', '').astype(int)
quick_current_ratio = (current_assets - inventory) / current_liabilities
BS_analysis['Acid test'] = quick_current_ratio


# Debt to equity

total_liabilities = BS_transpose['Total Liabilities'].str.replace(',', '').astype(int)
shareholders_equity = BS_transpose['Total stockholders\' equity'].str.replace(',', '').astype(int)
debt2equity_ratio = total_liabilities / shareholders_equity
BS_analysis['Debt to Equity Ratio'] = debt2equity_ratio
BS_analysis

# Rotacion cuentas por cobrar

net_credit_sales = IS_transpose['Net Income'].str.replace(',', '').astype(int)
average_net_receivables_for_the_period = BS_transpose['Net Receivables'].str.replace(',', '').astype(int)
receivable_turnover = net_credit_sales / average_net_receivables_for_the_period
BS_analysis['rot cxc'] = receivable_turnover

# Dias CxC

number_of_days_in_period = 360
average_age_of_receivables = number_of_days_in_period / receivable_turnover
BS_analysis['dias rot cxc'] = average_age_of_receivables


# Rotacion CxP
cost_sales = IS_transpose['Cost of Revenue'].str.replace(',', '').astype(int)
average_net_payables_for_the_period = BS_transpose['Accounts Payable'].str.replace(',', '').astype(int)
payable_turnover = cost_sales / average_net_payables_for_the_period
BS_analysis['rot cxp'] = payable_turnover

# Dias CxP
number_of_days_in_period = 360
average_age_of_payables = number_of_days_in_period / payable_turnover
BS_analysis['dias rot cxp'] = average_age_of_payables


#Rot Inventario
cost_of_goods_sold = IS_transpose['Cost of Revenue'].str.replace(',', '').astype(int)
average_inventory_for_the_period = inventory
inventory_turnover = cost_of_goods_sold / average_inventory_for_the_period
BS_analysis['rot inventario'] = inventory_turnover

# Dias Inventario
number_of_days_for_inventory_to_turn = number_of_days_in_period / inventory_turnover
BS_analysis['dias rot inventario'] = number_of_days_for_inventory_to_turn


# Ciclo de caja
ciclo_caja = number_of_days_for_inventory_to_turn + average_age_of_receivables - average_age_of_payables
BS_analysis['ciclo de caja'] = ciclo_caja

BS_analysis.to_excel("BS_google_analysis.xlsx")
BS_analysis

,Date,Working Capital,Current Ratio,Acid test,Debt to Equity Ratio,rot cxc,dias rot cxc,rot cxp,dias rot cxp,rot inventario,dias rot inventario,ciclo de caja
1,12/30/2019,107357000,3.374052,3.351960,0.369670,1.356037,265.479428,12.928610,27.845221,71.967968,5.002225,242.636433
2,12/30/2018,101056000,3.919006,3.887031,0.310559,1.648095,218.434033,16.422111,21.921665,64.946703,5.543007,202.055375
3,12/30/2017,100125000,5.140305,5.109333,0.293721,1.676265,214.763144,18.982786,18.964550,79.504673,4.528036,200.326630
4,12/30/2016,88652000,6.290762,6.274767,0.204702,0.895664,401.936503,22.333660,16.119167,170.085821,2.116579,387.933914


## Analysis: Income Statement

In [56]:
IS_analysis = pd.DataFrame(IS_transpose['Date']) # copy columns of dataframe


# margen bruto
revenue = IS_transpose['Total Revenue'].str.replace(',', '').astype(int)
cost_of_goods_sold = IS_transpose['Cost of Revenue'].str.replace(',', '').astype(int)
gross_profit_margin = (revenue - cost_of_goods_sold) / revenue
IS_analysis['margen bruto'] = gross_profit_margin * 100

# I+D contra ventas


RD_expense = IS_transpose['Research Development'].str.replace(',', '').astype(int)
RD_to_sales = RD_expense / revenue
IS_analysis['ID a ventas'] = RD_to_sales

# Margen operativo
operating_income = IS_transpose['Operating Income or Loss'].str.replace(',', '').astype(int)
operating_profit_margin = operating_income / revenue
IS_analysis['margen operativo'] = RD_to_sales

# Cobertura de intereses
interest_expense = IS_transpose['Interest Expense'].str.replace(',', '').astype(int)
earnings_before_interest_and_taxes = IS_transpose['Income Before Tax'].str.replace(',', '').astype(int)
interest_coverage_ratio = earnings_before_interest_and_taxes / interest_expense
IS_analysis['cobertura de intereses'] = interest_coverage_ratio

# Margen Neto
net_income = IS_transpose['Net Income'].str.replace(',', '').astype(int)
net_profit_margin = net_income / revenue
IS_analysis['margen neto'] = net_profit_margin

# ROE
net_profit = IS_transpose['Net Income available to common shareholders'].str.replace(',', '').astype(int)
average_shareholde_equity_for_the_period = BS_transpose['Total stockholders\' equity'].str.replace(',', '').astype(int)
return_on_equity = net_profit / average_shareholde_equity_for_the_period
IS_analysis['roe'] = return_on_equity
IS_analysis

# rotacion de activos
average_assets_for_the_period = BS_transpose['Total Assets'].str.replace(',', '').astype(int)
assets_turnover = revenue / average_assets_for_the_period
IS_analysis['rot activos'] = assets_turnover

# ROA
return_on_assets = net_profit_margin / assets_turnover
IS_analysis['Return on Assets'] = return_on_assets


IS_analysis.to_excel("IS_google_analysis.xlsx")
IS_analysis

,Date,margen bruto,ID a ventas,margen operativo,cobertura de intereses,margen neto,roe,rot activos,Return on Assets
1,ttm,55.580543,0.160747,0.160747,396.250000,0.212181,0.170486,0.586632,0.361694
2,12/30/2019,55.580543,0.160747,0.160747,396.250000,0.212181,0.193342,0.695286,0.305171
3,12/30/2018,56.476074,0.156550,0.156550,306.254386,0.224647,0.201545,0.693474,0.323944
4,12/30/2017,58.880520,0.149971,0.149971,249.477064,0.114221,0.091070,0.661833,0.172583
5,12/30/2016,61.075417,0.154511,0.154511,194.758065,0.215770,NaN,NaN,NaN


## Analysis: Cash Flow

In [63]:
CF_analysis = pd.DataFrame(CF_transpose['Date']) # copy columns of dataframe
CF_analysis

# Cobertura de pasivos

net_cash_provided_from_operating_activites = CF_transpose['Net cash provided by operating activites'].str.replace(',', '').astype(int)
average_current_liabilities = BS_transpose['Total Current Liabilities'].str.replace(',', '').astype(int)
current_liability_coverage_ratio = net_cash_provided_from_operating_activites / average_current_liabilities
CF_analysis['cobertura pasivos'] = current_liability_coverage_ratio

# precio de la accion/cash per share
share_price = 1103.37
operating_cash_flow = CF_transpose['Operating Cash Flow'].str.replace(',', '').astype(int)
common_stock = BS_transpose['Common Stock'].str.replace(',', '').astype(int)
operating_cash_flow_per_share = operating_cash_flow / common_stock
price_to_cash_flow_ratio = share_price /  operating_cash_flow_per_share
CF_analysis['Price to Cash Flow Ratio'] = price_to_cash_flow_ratio

CF_analysis.to_excel("CF_google_analysis.xlsx")

CF_analysis

,Date,cobertura pasivos,Price to Cash Flow Ratio
1,ttm,1.205635,1023.066035
2,12/30/2019,1.574812,911.696903
3,12/30/2018,1.983666,925.712042
4,12/30/2017,2.213595,1080.047844
5,12/30/2016,NaN,NaN


In [ ]:
IS_analysis.to_excel("IS_google_analysis.xlsx")